-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://camo.githubusercontent.com/290ee18049382f170e5f63956a389da081a8c16f/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f636f6d2e726176656e7061636b2e636d732f70616765732f6a702d6d6f7267616e2d6269672d646174612d61692d35343637413139302e6a7067"  style="width: 600px; height: 163px">
</div>

In [2]:
import requests
import os
from bs4 import BeautifulSoup
from azure.storage.blob import BlockBlobService

In [3]:
wasb_account_key = 'TODO'
wasb_account_name = 'TODO'

# Create the BlockBlockService that is used to call the Blob service for the storage account
block_blob_service = BlockBlobService(account_name=wasb_account_name, account_key=wasb_account_key)

# Create a container called 'quickstartblobs'.
container_name ='deepfin3'
if block_blob_service.exists(container_name) is False:
  block_blob_service.create_container(container_name)      

In [4]:
def scrape_find_zips(url):
    """ scrape the given url for any zip files in href tags """
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1',
    }

    response = requests.get(url, headers=headers)
    #print(response.status_code)
    #print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')
    all_tags = soup.find_all('a', href=True)
    tag_dict = {}
    for tag in all_tags:
      if '.zip' in tag['href']:
          tag_dict[tag.get_text()] = tag['href']
    return tag_dict


In [5]:
url = "https://www.asxhistoricaldata.com/archive/"

tag_dict = scrape_find_zips(url)

print(tag_dict.values())

In [6]:
from azure.storage.file import ContentSettings

def scrape_downloader(urls, dest):
    """ download all files listed in urls {} dict 
    Input:
        url = base url
        urls= target paths to append to base url
        dest= local path to store files
    """
    for k_name, v_path in urls.items():
        src = v_path        
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1',
        }
        r = requests.get(src, headers=headers)  
        if r is not None:
          block_blob_service.create_blob_from_bytes(dest, k_name + '.zip', r.content)            
          print(src)

In [7]:
scrape_downloader(tag_dict, container_name)

In [8]:
blobs = block_blob_service.list_blobs(container_name)

for blob in blobs:
  print(blob.name)

In [9]:
import zipfile
import csv
import datetime
from io import BytesIO, TextIOWrapper
from pyspark.sql import Row


#get all files in container
f_zip = [ blob.name for blob in blobs]

#Create empty lists
newlines = lines = header = all_lines = []

#limit to single 
for f in f_zip: #[:1]: limit
    #only go through zip files in container 
    if f.endswith(".zip"):
      f_bytes = block_blob_service.get_blob_to_bytes(container_name, f)
      with zipfile.ZipFile(BytesIO(f_bytes.content)) as main_zip:  
        for inner_file in main_zip.namelist():   
            #print(inner_file)
            #read inner zip file
            zfiledata = BytesIO(main_zip.read(inner_file))
            inner_csv = TextIOWrapper(zfiledata, encoding='utf8', newline='')            
            reader = csv.reader(inner_csv, delimiter=',', dialect=csv.unix_dialect)
            
            for row, line in enumerate(reader):
                #print(line)
                # 'Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'
                lines.append(Row(Ticker=line[0], Date=datetime.datetime.strptime(line[1], '%Y%m%d') , High=float(line[2]), Low=float(line[3]), Close=float(line[4]), Volume=float(line[5])))
            

In [10]:
df = sqlContext.createDataFrame(sc.parallelize(lines))

In [11]:
df.printSchema()

In [12]:
df.count()

In [13]:
%sql drop table if exists asxhistoric;

In [14]:
df.write.saveAsTable("asxhistoric")

In [15]:
from pyspark.sql.functions import year

focus_df = ((df 
  .select("Date", "Close", "Ticker") 
  .filter("Ticker = 'AAA'")
  .filter(year("Date") == '1998')
))

In [16]:
focus_df.count()

In [17]:
display(focus_df)

In [18]:
%sql drop table if exists timeseries;

In [19]:
focus_df.write.saveAsTable("timeseries")

In [20]:
from pyspark.sql.functions import year

df_2016 = ((df 
  .filter(year("Date") == '2016')
))

In [21]:
%sql drop table if exists asxhistoric2016;

In [22]:
df_2016.write.saveAsTable("asxhistoric2016")

In [23]:
%sql
SELECT COUNT(*) FROM asxhistoric2016;